## The plan for the hybrid model:

1. Write in python sams hybrid model and demonstrate learning
2. Fix the tuned params as ground truth and apply stan to learn them
3. demonstrate learning using groud truth graphed with error lines
4. Try post hoc learning?